<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **ECCOv4 from NASA Earth Data Cloud**<span style='color:#0066cc'> 

<img src="img/ECCOv4.png" alt="drawing" width="750"/>    


<span style='color:#0066cc'><font size="5"> **About the "Estimating the Circulation and Climate of the Ocean" [ECCO](https://ecco-group.org/) consortium**
1. <font size="3"><span style='color:Black'> Uses a combination of assimilated data from (various sources) to constrain the simulated global ocean and atmosphere model.
3. <font size="3"><span style='color:Black'> Remains widely used by climate scientists.
4. <font size="3"><span style='color:Black'> ECCO belongs to a hierarchy of global models that all share the same grid topology (Cube Sphere), but differ in horizontal resolution. 



<span style='color:#ff6666'><font size="5">**Requirements**
1. <font size="3"><span style='color:Black'> Have a Bearer Token for EarthData in the Cloud (See `GetStarted` Notebook).
2. <font size="3"><span style='color:Black'> Upload the Bearer Token from local file`token.json`



 <span style='color:#ff6666'><font size="5"> **Objectives**
 
<font size="3"><span style='color:Black'> Use [pydap](https://pydap.github.io/pydap/)'s client API to:
 
- <font size="3"><span style='color:Black'> Understand `Dimensions`, `Groups`, `Maps` and `Arrays` in the DAP4 model.
- <font size="3"><span style='color:Black'> Develop understanding of metadata of remote files accessed via OPeNDAP.
- <font size="3"><span style='color:Black'> How to integrate a workflow involving multiple OPeNDAP URLs and xarray parallelism, lazy evaluation, and
- <font size="3"><span style='color:Black'> To demonstrate an advanced workflow for remote access and plotting of **Level 4** with complex Topology ECCOv4 Data via OPeNDAP



<span style='color:#ff6666'><font size="5">**Browsing Data**:

<font size="3"><span style='color:Black'> Broad information about the dataset can be found in the PODAAC website (see [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO)).


<font size="3"><span style='color:Black'> Some variables or interest are:

- <font size="3"> [Native grid](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4)
- <font size="3"> [Temperature and Salinity](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4)
- <font size="3"> [Velocities](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_OCEAN_VEL_LLC0090GRID_MONTHLY_V4R4)
- <font size="3"> [Mixed layer depth](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_MIXED_LAYER_DEPTH_LLC0090GRID_MONTHLY_V4R4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from pydap.client import open_url
import json
import cartopy.crs as ccrs
import xarray as xr

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Access EARTHDATA**

<font size="3"><span style='color:Black'> Many of the data variables can be browsed [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO). Two types of data can be found:

1. <font size="3"> Interpolated data onto a regular Lat-Lon grid.
2. <font size="3"> Data on the native LLC (lat-lon-cap) grid.

<font size="3"><span style='color:Black'> Here we will work with the original data defined on the Lat-Lon-Cap (LLC90) grid.

In [ ]:
Grid_url = 'dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Geometry%20Parameters%20for%20the%20Lat-Lon-Cap%2090%20(llc90)%20Native%20Model%20Grid%20(Version%204%20Release%204)/granules/GRID_GEOMETRY_ECCO_V4r4_native_llc0090'

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 


In [ ]:
# load token json data
with open('token.json', 'r') as fp:
    Authorization = json.load(fp)

# pass Token Authorization to a new Session.
my_session = requests.Session()
my_session.headers = Authorization


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Lazy access to remote data via pydap's client API**

<font size="3"> pydap exploits the OPeNDAP's separation between metadata and data, to create lazy dataset objects that point to the data. These lazy objects contain all the attributes detailed in OPeNDAP's metadata files (DMR)

In [ ]:
%%time
ds_grid = open_url(Grid_url, session=my_session, protocol="dap4")

In [ ]:
ds_grid.tree()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Lets play with some data!**

<font size="3.5"> To download remote OPeNDAP-hosted data via pydap, the syntax is:

```python
# this fetches remote data into a pydap object container
VarName = dataset[<varName>][:]
```

<font size="3.5"> This downloads data into "local" memory (RAM) as (uncompressed) numpy arrays. Data lies within a pydap object, and can be made avaiable into an array using the following syntax:

```python
# The `.data` command allows direct access to the Numpy array (e.g. for manipulation)
VarName.data
```



In [ ]:
%%time
# lets download some data
Depth = ds_grid['Depth'][:]
print(type(Depth))

In [ ]:
Depth.attributes

In [ ]:
Depth.shape, Depth.dimensions

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot Depth along native grid**

<font size="3.5">ECCO data is defined on a Cube Sphere -- meaning the horizontal grid contains an `extra` dimension: `tile` or `face`.


<font size="3.5"> Lets plots all the horizontal data natively by doing a projection of the Cube-Sphere onto the plane:


In [ ]:
%%time
Variable = [Depth[i].data for i in range(13)]
clevels =  np.linspace(0, 6000, 100)
cMap = 'Greys_r'

In [ ]:
%%time
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES = [
    axes[4, 0], axes[3, 0], axes[2, 0], axes[4, 1], axes[3, 1], axes[2, 1],
    axes[1, 1], 
    axes[1, 2], axes[1, 3], axes[1, 4], axes[0, 2], axes[0, 3], axes[0, 4],
]
for i in range(len(AXES)):
    AXES[i].contourf(Variable[i], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot with corrected Topology**

In [ ]:
%%time
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], clevels, cmap=cMap)

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], clevels, cmap=cMap)

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

<span style='color:#ff6666'><font size="5.5">**Exercise:**

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot another Variable**

- <font size="3"> Inspect attributes to understand what these variables, their units, and any missing value.
- <font size="3"> <font size="3"> Create new Variable and clevels arrays and colormap. Something like this:
```python
VarName = ds_grid[<varName>][:]
Variable = [VarName[i].data for i in range(13)]
cMap = <new_colormap> # see https://matplotlib.org/stable/users/explain/colors/colormaps.html
```



<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **pydap + xarray to aggregate multiple URLs**


<font size="3.5"> OPeNDAP allows remote access via `dataURL`s. Each `dataURL` represents a variable, i.e. a piece of the whole puzzle. We can exploit `xarray` concatenation and parallelism to combine multiple `dataURL`s, and thus multiple pydap objects, into a single `xarray.Dataset`. Can do this because <span style='color:#ff6666'>**xarray has long-implemented pydap as a backend**




<span style='font-family:serif'> <font size="5"><span style='color:#0066cc'> **Lets start with one dataURL**

<font size="3.5"> and decode the dataURL so that later we can iterate over multiple variables, multiple years. 

<font size="3.5"> The only modification we will to do is replace `http` with `dap4` in the `dataURL`. While this is **NOT strictly necessary**, it encodes the DAP4 protocol within pydap without adding extra arguments. This is then later passed onto `xarray`.


```python
basURL = "dap4://..." # this, instead of http:// specifies the DAP4 protocol without any extra arguments
```



In [ ]:
baseURL = 'dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/'
Temp_Salt = "ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%20llc90%20Grid%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_"
year = '2017-'
month = '01'
end_ = '_ECCO_V4r4_native_llc0090'
Temp_2017 = baseURL + Temp_Salt +year  + month + end_


In [ ]:
%%time
temp_ds = open_url(Temp_2017, session=my_session)

In [ ]:
temp_ds.tree()

In [ ]:
temp_ds['THETA'].shape, temp_ds['THETA'].dimensions

In [ ]:
print('Dataset memory used [GBs, uncompressed]: ', temp_ds.nbytes/1e9)

In [ ]:
print('Array memory used [GBs, uncompressed]: ', temp_ds['THETA'].nbytes/1e9)

In [ ]:
%%time
dataset = xr.open_dataset(Temp_2017, engine='pydap', session=my_session)
dataset

<font size="3.5"> <span style='color:#ff6666'> **Question: <span style='color:#0066cc'> How does it compare to open a remote dataset via pydap vs via xarray?**

<span style='color:black'><font size="3.5"> Any difference may result from ``xarray``'s much specified methods for decoding all variables listed, and metadata interface that has become a standard in many python packages.

<font size="3.5"> Lets access the minimal amount of data variables. We can so this via OPeNDAP's URL


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Add a Constraint expression to the URL to ONLY retrieve THETA**

<font size="3"> You can do that by appending to the <base_URL> the CE syntax:

```python
<base_URL>?dap4.ce=/<VarName>
```

<font size="3"> where `<VarName>` is the name of the Array you want to keep

In [ ]:
%%time
temp_ds = open_url(Temp_2017+'?dap4.ce=/THETA', session=my_session)
temp_ds

In [ ]:
temp_ds.tree()

<font size="5"><span style='color:#0066cc'> **NOTE**<span style='color:black'> with xarray - need to include all dimensions associated with THETA in the constraint expression.


In [ ]:
%%time
CE = '?dap4.ce=/THETA;/SALT;/tile;/j;/k;/i;/time'
dataset = xr.open_dataset(Temp_2017+CE, engine='pydap', session=my_session)

<font size="5"><span style='color:#0066cc'> **Create a list of URLs that spans the entire 2017**<span style='color:black'> 
```python
[f'-{i:02}' for i in range(1, 13)]
```

<font size="3.5"> The relevant variables in this case are:

```python
baseURL
Temp_Salt
year
end_
CE
```



In [ ]:
Temp_2017 = [baseURL + Temp_Salt + year + f'{i:02}' + end_ + CE for i in range(1, 13)]

In [ ]:
%%time
theta_salt_ds = xr.open_mfdataset(Temp_2017, engine='pydap', session=my_session, parallel=True, chunks={'time':1})
theta_salt_ds

<span style='color:#ff6666'><font size="5.5">**Exercise:**

- <font size="3.5"><span style='color:black'> Create an `xarray.Dataset` for Temperature (`THETA`) and Salinity (`SALT`) for multiple years and months available
- <font size="3.5"><span style='color:black'> Make some surface maps to visualize these variables in the native and corrected topology. 


<font size="3.5"> **HINT** To use the plot function above, you will need to create new `Variable` and `clevel` from the `xarray.Dataset`. Something like this:
```python
# this is an example for a 4Dimensional array
Variable = [xarray.Dataset[<varName>].isel(time=0, k=0, tile=i) for i in range(13)]
clevels =  np.linspace(<vmin>, <vmax>, 100)
cMap = <new_colormap> # see https://matplotlib.org/stable/users/explain/colors/colormaps.html
```